In [1]:
import warnings 
import holidays
import numpy as np
import pandas as pd
import xgboost as xgb
import seaborn as sns
import statsmodels.api as sm
import matplotlib.pyplot as plt
import itertools
from prophet import Prophet
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.arima.model import ARIMA
import multiprocessing
from joblib import Parallel, delayed
plt.style.use('ggplot')

from xgboost import XGBRegressor
from IPython.display import display
from matplotlib.pyplot import figure
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasRegressor
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, make_scorer
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from sklearn.model_selection import RandomizedSearchCV
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.tsa.statespace.sarimax import SARIMAX
from math import sqrt

warnings.filterwarnings('ignore')

Importing plotly failed. Interactive plots will not work.


In [ ]:
# Holt Winters
decomposition = seasonal_decompose(y_train, model='mul', period=24)
fig, axs = plt.subplots(4, 1, figsize=(12, 10), sharex=True)
decomposition.observed.plot(ax=axs[0], title='Observed')
decomposition.trend.plot(ax=axs[1], title='Trend')
decomposition.seasonal.plot(ax=axs[2], title='Seasonal')
decomposition.resid.plot(ax=axs[3], title='Residual')
plt.tight_layout()
plt.show()

In [ ]:
model = ExponentialSmoothing(y_train, 
    trend='add',  
    seasonal='add',
    seasonal_periods=24,  # For hourly data
    damped=True,  
    initialization_method="estimated"
)


hw_model = model.fit(optimized=True)
predictions = hw_model.forecast(len(y_valid))

In [ ]:
# Evaluate the model
rmse = sqrt(mean_squared_error(y_valid, predictions))
mape = mean_absolute_percentage_error(y_valid, predictions) * 100 
print("Model Evaluation Metrics:")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")

# Plot results with error metrics in title - showing only 100 points
plt.figure(figsize=(16, 8), dpi=100)

# Get the last 100 points of predictions and validation data
last_100_predictions = predictions[-100:]
last_100_actual = y_valid[-100:]

# Plot only these 100 points
plt.plot(last_100_actual, label='Actual Values', color='green')
plt.plot(last_100_predictions, label='Holt-Winters Forecast', color='blue')

plt.title(f'Holt-Winters Forecast - Last 100 Points (RMSE: {rmse:.2f}, MAPE: {mape:.2f}%)')
plt.xlabel('Date')
plt.ylabel('Value')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Get model parameters
print("\nModel Parameters:")
print(f"Alpha (level): {hw_model.params['smoothing_level']:.4f}")
print(f"Beta (trend): {hw_model.params['smoothing_trend']:.4f}")
print(f"Gamma (seasonality): {hw_model.params['smoothing_seasonal']:.4f}")

In [ ]:
plt.subplot(4, 1, 2)
plt.plot(hw_model.level, label='Level')
plt.legend()
plt.title('Level Component')

plt.subplot(4, 1, 3)
plt.plot(hw_model.trend, label='Trend')
plt.legend()
plt.title('Trend Component')

plt.subplot(4, 1, 4)
plt.plot(hw_model.season, label='Seasonality')
plt.legend()
plt.title('Seasonal Component')

plt.tight_layout()
plt.show()

In [ ]:
# Calculate and plot forecast error
plt.figure(figsize=(12, 6))
forecast_errors = y_valid.values - predictions.values
plt.plot(y_valid.index, forecast_errors)
plt.axhline(y=0, color='r', linestyle='-')
plt.title('Forecast Errors (Actual - Predicted)')
plt.xlabel('Date')
plt.ylabel('Error')
plt.tight_layout()
plt.show()

# Plot error distribution
plt.figure(figsize=(10, 6))
plt.hist(forecast_errors, bins=30, alpha=0.7)
plt.axvline(x=np.mean(forecast_errors), color='r', linestyle='--', label=f'Mean: {np.mean(forecast_errors):.2f}')
plt.title('Error Distribution')
plt.xlabel('Forecast Error')
plt.ylabel('Frequency')
plt.legend()
plt.tight_layout()
plt.show()